In [23]:
import numpy as np
import pandas as pd
import os
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings

In [24]:
warnings.filterwarnings('ignore')

In [25]:
os.getcwd()

'C:\\Users\\hamma\\Desktop\\MAIN\\ElectricityPriceForecastingWithNeuralNetworks\\Data'

In [26]:
day_ahead_2yr = pd.read_csv('Day-ahead_prices_201901010000_202101012359.csv', delimiter=';', usecols=(0,1,5))
day_ahead_rest = pd.read_csv('Day-ahead_prices_202101020000_202110252359.csv', delimiter=';', usecols=(0,1,5))
day_ahead_price_all = pd.concat([day_ahead_2yr, day_ahead_rest],ignore_index=True)
day_ahead_price_all.columns = ['Date', 'TimeOfDay', 'France___MWh_']

In [27]:
newRows = {'Date': ["Mar 31, 2019","Mar 29, 2020","Mar 28, 2021","Oct 27, 2019","Oct 25, 2020"],
        'TimeOfDay': ["2:00 AM","2:00 AM","2:00 AM","2:00 AM","2:00 AM"],
       'France___MWh_':[day_ahead_price_all.France___MWh_[2137:2139].mean(),
                        day_ahead_price_all.France___MWh_[10873:10875].mean(),
                       day_ahead_price_all.France___MWh_[19609:19611].mean(),
                        day_ahead_price_all.France___MWh_[7177:7179].mean(),
                       day_ahead_price_all.France___MWh_[15913:15915].mean()]}
newRows = pd.DataFrame.from_dict(newRows,orient='columns')

In [28]:
final_data_day_ahead = pd.concat([day_ahead_price_all[0:2138],newRows[0:1],
                                 day_ahead_price_all[2138:7177],newRows[3:4],
                                 day_ahead_price_all[7179:10874],newRows[1:2],
                                 day_ahead_price_all[10874:15913],newRows[4:5],
                                 day_ahead_price_all[15915:19610],newRows[2:3],
                                 day_ahead_price_all[19610::]],ignore_index=True)


In [29]:
final_day_ahead_demand_all = pd.read_csv('FRhourlyload.csv', delimiter=',',parse_dates=['Time (CET/CEST)'])
final_day_ahead_demand_all.columns = ['Date', 'DayAheadDemand']
data = pd.concat([final_day_ahead_demand_all['Date'],
                  final_data_day_ahead['France___MWh_'],
                  final_day_ahead_demand_all['DayAheadDemand']],axis=1,ignore_index=False)

In [30]:
days = [1,2,3,4,5,6,0]*1029
Weekday = pd.DataFrame(np.zeros((24696, 1)))
idx = 0
for i in days:
    Weekday[idx:idx+24] = i
    idx = idx + 24
data['Weekday'] = Weekday  

mincol = np.zeros(24720)
for i in range(24,24721,24):
    mincol[i:i+24] = data['France___MWh_'][i-24:i].min()
mincol = pd.DataFrame(mincol[24:24721])
data = pd.concat([data,mincol],axis=1,ignore_index=False)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol']


In [31]:
hr_24_prev_day = np.zeros(24720)
for i in range(24,24721,24):
    hr_24_prev_day[i:i+24] = data['DayAheadPrice'][i-1:i]
hr_24_prev_day = pd.DataFrame(hr_24_prev_day[24:24721])
data = pd.concat([data,hr_24_prev_day],axis=1,ignore_index=True)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol','24hrbefore']
data

,Date,DayAheadPrice,DayAheadDemand,Weekday,MinCol,24hrbefore
0,2019-01-01 00:00:00,51.00,63450.0,1.0,20.90,54.91
1,2019-01-01 01:00:00,46.27,60800.0,1.0,20.90,54.91
2,2019-01-01 02:00:00,39.78,59950.0,1.0,20.90,54.91
3,2019-01-01 03:00:00,27.87,56400.0,1.0,20.90,54.91
4,2019-01-01 04:00:00,23.21,53550.0,1.0,20.90,54.91
...,...,...,...,...,...,...
24691,2021-10-25 19:00:00,262.00,60450.0,0.0,93.09,205.47
24692,2021-10-25 20:00:00,250.00,58350.0,0.0,93.09,205.47
24693,2021-10-25 21:00:00,224.23,53800.0,0.0,93.09,205.47
24694,2021-10-25 22:00:00,219.63,51750.0,0.0,93.09,205.47


In [32]:
dumsat = np.zeros(24696)
dumsun = np.zeros(24696)
dummon = np.zeros(24696)

for i in range(24696):
    if data['Weekday'][i] == 5:
        dumsat[i] = 1
    if data['Weekday'][i] == 6:
        dumsun[i] = 1   
    if data['Weekday'][i] == 0:
        dummon[i] = 1
        
dumsat = pd.DataFrame(dumsat)
dumsun = pd.DataFrame(dumsun)
dummon = pd.DataFrame(dummon)

data = pd.concat([data,dumsat,dumsun,dummon],axis=1,ignore_index=False)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol','24hrbefore','DummSat','DummSun','DummMon']
data

,Date,DayAheadPrice,DayAheadDemand,Weekday,MinCol,24hrbefore,DummSat,DummSun,DummMon
0,2019-01-01 00:00:00,51.00,63450.0,1.0,20.90,54.91,0.0,0.0,0.0
1,2019-01-01 01:00:00,46.27,60800.0,1.0,20.90,54.91,0.0,0.0,0.0
2,2019-01-01 02:00:00,39.78,59950.0,1.0,20.90,54.91,0.0,0.0,0.0
3,2019-01-01 03:00:00,27.87,56400.0,1.0,20.90,54.91,0.0,0.0,0.0
4,2019-01-01 04:00:00,23.21,53550.0,1.0,20.90,54.91,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
24691,2021-10-25 19:00:00,262.00,60450.0,0.0,93.09,205.47,0.0,0.0,1.0
24692,2021-10-25 20:00:00,250.00,58350.0,0.0,93.09,205.47,0.0,0.0,1.0
24693,2021-10-25 21:00:00,224.23,53800.0,0.0,93.09,205.47,0.0,0.0,1.0
24694,2021-10-25 22:00:00,219.63,51750.0,0.0,93.09,205.47,0.0,0.0,1.0


In [33]:
data = data.to_numpy()
data = data[:, [1,2,3,4,5,6,7,8]]
cal_len = 731
test_len = data.shape[0] // 24 - cal_len
real = data[-298*24:, 0]
data = data.astype('float64')
data

array([[5.1000e+01, 6.3450e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [4.6270e+01, 6.0800e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.9780e+01, 5.9950e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [2.2423e+02, 5.3800e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [2.1963e+02, 5.1750e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [2.0547e+02, 5.4000e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [34]:
import tensorflow
from tensorflow import keras
import numpy as np
import optuna 
import logging
import sys
cal_len = 731
test_len = data.shape[0] // 24 - cal_len
real = data[-298*24:, 0].reshape((298, 24))
price = data[:731*24, 0].reshape((731, 24))
load = data[:731*24, 1].reshape((731, 24))
#dumm = data[:731*24:, 2].reshape((731, 24))
minn = data[:731*24, 3].reshape((731, 24))
prev24 = data[:731*24,4].reshape((731, 24))
dumsat = data[:731*24, 5].reshape((731, 24))
dumsun = data[:731*24, 6].reshape((731, 24))
dummon = data[:731*24, 7].reshape((731, 24))

In [35]:
real_train = data[:731*24, 0]

y = real_train.reshape((731, 24))
y = y[7:]

x = np.hstack([price[6:-1, :], price[:-7, :], load[7:, :], minn[6:-1, 0:1],prev24[6:-1, 0:1],
              dumsat[7:, 0:1], dumsun[7:, 0:1],dummon[7:, 0:1]])

#x= x.T
#x = x.reshape((724, 24*9))

pricef = data[:, 0].reshape((1029, 24))
loadf = data[:, 1].reshape((1029, 24))
minnf = data[:, 3].reshape((1029, 24))
prev24f = data[:,4].reshape((1029, 24))
dumsatf = data[:, 5].reshape((1029, 24))
dumsunf = data[:, 6].reshape((1029, 24))
dummonf = data[:, 7].reshape((1029, 24))

xf = np.hstack([pricef[730:-1, :], pricef[724:-7, :], loadf[731:, :],
                   minnf[730:-1, 0:1], prev24f[731:, 0:1],
                  dumsatf[731:, 0:1], dumsunf[731:, 0:1], dummonf[731:, 0:1]])
#xf = xf.T
#xf = xf.reshape((298, 24*9))


permutation = np.random.permutation(list(range(724)))
percentage_val = 0.2
Nval = int(724 * percentage_val)
seen_x = x[permutation[:724-Nval], :]
seen_y = y[permutation[:724-Nval], :]
unseen_x = x[permutation[724-Nval:], :]
unseen_y = y[permutation[724-Nval:], :]


In [36]:
price[6:-1, :].shape

(724, 24)

In [37]:
def objective(trial):
    inputs = keras.Input((77)) # Input layer of size equal to number of columns
    norm = keras.layers.BatchNormalization()(inputs)
    hidden = keras.layers.Dense(trial.suggest_int('hidden_neurons', 100, 500), activation='sigmoid')(norm)
    hidden2 = keras.layers.Dense(trial.suggest_int('hidden_neurons2', 100, 500), activation = 'sigmoid')(hidden)
    output = keras.layers.Dense(24, activation='linear')(hidden2)
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    model.fit(seen_x, seen_y, epochs=50, verbose=0, batch_size=32)
    pred = model.predict(unseen_x)
    return (np.mean(np.abs(pred - unseen_y)))

In [39]:
optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
study_name = 'TEST'
storage_name = f'sqlite:///trials/{study_name}'
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=False)
study.optimize(objective, n_trials=10, show_progress_bar=True)
best_params = study.best_params
print(best_params)
print(study.trials_dataframe())

[I 2023-03-25 22:51:25,432] A new study created in RDB with name: TEST


A new study created in RDB with name: TEST
A new study created in RDB with name: TEST


  0%|          | 0/10 [00:00<?, ?it/s]

5/5 [==============================] - 0s 3ms/step
Trial 0 finished with value: 4.999252584960174 and parameters: {'hidden_neurons': 436, 'hidden_neurons2': 275}. Best is trial 0 with value: 4.999252584960174.
Trial 0 finished with value: 4.999252584960174 and parameters: {'hidden_neurons': 436, 'hidden_neurons2': 275}. Best is trial 0 with value: 4.999252584960174.


 10%|█         | 1/10 [00:04<00:37,  4.12s/it]

...

[I 2023-03-25 22:51:52,265] Trial 7 finished with value: 4.770286233353393 and parameters: {'hidden_neurons': 495, 'hidden_neurons2': 415}. Best is trial 3 with value: 4.690096357961496.
5/5 [==============================] - 0s 2ms/step
Trial 8 finished with value: 4.743942529503946 and parameters: {'hidden_neurons': 401, 'hidden_neurons2': 333}. Best is trial 3 with value: 4.690096357961496.
Trial 8 finished with value: 4.743942529503946 and parameters: {'hidden_neurons': 401, 'hidden_neurons2': 333}. Best is trial 3 with value: 4.690096357961496.


 90%|█████████ | 9/10 [00:29<00:03,  3.31s/it]

[I 2023-03-25 22:51:55,229] Trial 8 finished with value: 4.743942529503946 and parameters: {'hidden_neurons': 401, 'hidden_neurons2': 333}. Best is trial 3 with value: 4.690096357961496.
5/5 [==============================] - 0s 2ms/step
Trial 9 finished with value: 4.926601490830933 and parameters: {'hidden_neurons': 467, 'hidden_neurons2': 212}. Best is trial 3 with value: 4.690096357961496.
Trial 9 finished with value: 4.926601490830933 and parameters: {'hidden_neurons': 467, 'hidden_neurons2': 212}. Best is trial 3 with value: 4.690096357961496.


100%|██████████| 10/10 [00:32<00:00,  3.28s/it]

[I 2023-03-25 22:51:58,247] Trial 9 finished with value: 4.926601490830933 and parameters: {'hidden_neurons': 467, 'hidden_neurons2': 212}. Best is trial 3 with value: 4.690096357961496.
{'hidden_neurons': 468, 'hidden_neurons2': 447}
   number     value             datetime_start          datetime_complete  \
0       0  4.999253 2023-03-25 22:51:25.485543 2023-03-25 22:51:29.500848   
1       1  4.973492 2023-03-25 22:51:29.596160 2023-03-25 22:51:33.292225   
2       2  5.017250 2023-03-25 22:51:33.355552 2023-03-25 22:51:35.955844   
3       3  4.690096 2023-03-25 22:51:36.094508 2023-03-25 22:51:39.197990   
4       4  5.541790 2023-03-25 22:51:39.289124 2023-03-25 22:51:41.854622   
5       5  5.127702 2023-03-25 22:51:41.947805 2023-03-25 22:51:44.628836   
6       6  5.417841 2023-03-25 22:51:44.726939 2023-03-25 22:51:47.891582   
7       7  4.770286 2023-03-25 22:51:47.971055 2023-03-25 22:51:52.175900   
8       8  4.743943 2023-03-25 22:51:52.285151 2023-03-25 22:51:55.15018

In [40]:
study.best_params

{'hidden_neurons': 468, 'hidden_neurons2': 447}

In [41]:
def evaluate(params):
    inputs = keras.Input((77))
    norm = keras.layers.BatchNormalization()(inputs)
    hidden = keras.layers.Dense(params['hidden_neurons'], activation="sigmoid")(norm)
    hidden2 = keras.layers.Dense(params['hidden_neurons2'], activation="sigmoid")(hidden)
    output = keras.layers.Dense(24, activation='linear')(hidden2)
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    model.fit(x, y, epochs=500, verbose=2, batch_size=32)
    preds = model.predict(xf)
    hr_analysis = []
    for hour in range(24):
        real_hour = real[hour::24]
        preds_hour = preds[hour::24]
        row = [(np.mean(np.abs(preds_hour - real_hour))), sqrt(mean_squared_error(real_hour, preds_hour)) ]
        hr_analysis.append(row)
    df = pd.DataFrame(hr_analysis)
    return(np.mean(np.abs(preds - real))), sqrt(mean_squared_error(real, preds)),df, preds

In [42]:
evaluate(study.best_params)

Epoch 1/500
23/23 - 0s - loss: 1194.1899 - 482ms/epoch - 21ms/step
Epoch 2/500
23/23 - 0s - loss: 740.4514 - 96ms/epoch - 4ms/step
Epoch 3/500
23/23 - 0s - loss: 440.8526 - 88ms/epoch - 4ms/step
Epoch 4/500
23/23 - 0s - loss: 274.5625 - 101ms/epoch - 4ms/step
Epoch 5/500
23/23 - 0s - loss: 185.4086 - 90ms/epoch - 4ms/step
Epoch 6/500
23/23 - 0s - loss: 142.5647 - 104ms/epoch - 5ms/step
Epoch 7/500
23/23 - 0s - loss: 125.1936 - 92ms/epoch - 4ms/step
Epoch 8/500
23/23 - 0s - loss: 113.9610 - 125ms/epoch - 5ms/step
Epoch 9/500
23/23 - 0s - loss: 107.2990 - 99ms/epoch - 4ms/step
Epoch 10/500
23/23 - 0s - loss: 101.6211 - 102ms/epoch - 4ms/step
Epoch 11/500
23/23 - 0s - loss: 92.5013 - 87ms/epoch - 4ms/step
Epoch 12/500
23/23 - 0s - loss: 86.6118 - 105ms/epoch - 5ms/step
Epoch 13/500
23/23 - 0s - loss: 82.6591 - 85ms/epoch - 4ms/step
Epoch 14/500
23/23 - 0s - loss: 80.9411 - 89ms/epoch - 4ms/step
Epoch 15/500
23/23 - 0s - loss: 75.9550 - 95ms/epoch - 4ms/step
Epoch 16/500
23/23 - 0s - loss:

23/23 - 0s - loss: 35.6135 - 120ms/epoch - 5ms/step
Epoch 128/500
23/23 - 0s - loss: 34.4745 - 120ms/epoch - 5ms/step
Epoch 129/500
23/23 - 0s - loss: 36.1862 - 100ms/epoch - 4ms/step
Epoch 130/500
23/23 - 0s - loss: 35.3010 - 114ms/epoch - 5ms/step
Epoch 131/500
23/23 - 0s - loss: 34.7839 - 101ms/epoch - 4ms/step
Epoch 132/500
23/23 - 0s - loss: 36.4439 - 114ms/epoch - 5ms/step
Epoch 133/500
23/23 - 0s - loss: 34.4481 - 118ms/epoch - 5ms/step
Epoch 134/500
23/23 - 0s - loss: 35.6441 - 102ms/epoch - 4ms/step
Epoch 135/500
23/23 - 0s - loss: 36.7040 - 122ms/epoch - 5ms/step
Epoch 136/500
23/23 - 0s - loss: 37.5356 - 127ms/epoch - 6ms/step
Epoch 137/500
23/23 - 0s - loss: 32.9479 - 106ms/epoch - 5ms/step
Epoch 138/500
23/23 - 0s - loss: 36.1053 - 114ms/epoch - 5ms/step
Epoch 139/500
23/23 - 0s - loss: 33.5657 - 108ms/epoch - 5ms/step
Epoch 140/500
23/23 - 0s - loss: 38.7594 - 105ms/epoch - 5ms/step
Epoch 141/500
23/23 - 0s - loss: 34.2335 - 104ms/epoch - 5ms/step
Epoch 142/500
23/23 - 0s

Epoch 252/500
23/23 - 0s - loss: 28.2293 - 111ms/epoch - 5ms/step
Epoch 253/500
23/23 - 0s - loss: 26.1201 - 96ms/epoch - 4ms/step
Epoch 254/500
23/23 - 0s - loss: 31.8991 - 109ms/epoch - 5ms/step
Epoch 255/500
23/23 - 0s - loss: 29.3185 - 101ms/epoch - 4ms/step
Epoch 256/500
23/23 - 0s - loss: 27.7384 - 120ms/epoch - 5ms/step
Epoch 257/500
23/23 - 0s - loss: 26.8637 - 118ms/epoch - 5ms/step
Epoch 258/500
23/23 - 0s - loss: 27.6806 - 101ms/epoch - 4ms/step
Epoch 259/500
23/23 - 0s - loss: 26.2934 - 110ms/epoch - 5ms/step
Epoch 260/500
23/23 - 0s - loss: 27.5133 - 99ms/epoch - 4ms/step
Epoch 261/500
23/23 - 0s - loss: 29.0337 - 107ms/epoch - 5ms/step
Epoch 262/500
23/23 - 0s - loss: 28.8276 - 100ms/epoch - 4ms/step
Epoch 263/500
23/23 - 0s - loss: 24.7825 - 113ms/epoch - 5ms/step
Epoch 264/500
23/23 - 0s - loss: 27.3788 - 95ms/epoch - 4ms/step
Epoch 265/500
23/23 - 0s - loss: 26.9386 - 115ms/epoch - 5ms/step
Epoch 266/500
23/23 - 0s - loss: 27.4837 - 114ms/epoch - 5ms/step
Epoch 267/500

Epoch 377/500
23/23 - 0s - loss: 21.5522 - 110ms/epoch - 5ms/step
Epoch 378/500
23/23 - 0s - loss: 22.0266 - 93ms/epoch - 4ms/step
Epoch 379/500
23/23 - 0s - loss: 22.1843 - 98ms/epoch - 4ms/step
Epoch 380/500
23/23 - 0s - loss: 21.1980 - 90ms/epoch - 4ms/step
Epoch 381/500
23/23 - 0s - loss: 20.5640 - 115ms/epoch - 5ms/step
Epoch 382/500
23/23 - 0s - loss: 20.3159 - 98ms/epoch - 4ms/step
Epoch 383/500
23/23 - 0s - loss: 20.9703 - 108ms/epoch - 5ms/step
Epoch 384/500
23/23 - 0s - loss: 21.3649 - 94ms/epoch - 4ms/step
Epoch 385/500
23/23 - 0s - loss: 21.9210 - 113ms/epoch - 5ms/step
Epoch 386/500
23/23 - 0s - loss: 23.9212 - 104ms/epoch - 5ms/step
Epoch 387/500
23/23 - 0s - loss: 20.5171 - 106ms/epoch - 5ms/step
Epoch 388/500
23/23 - 0s - loss: 22.1574 - 91ms/epoch - 4ms/step
Epoch 389/500
23/23 - 0s - loss: 20.3471 - 123ms/epoch - 5ms/step
Epoch 390/500
23/23 - 0s - loss: 23.6097 - 105ms/epoch - 5ms/step
Epoch 391/500
23/23 - 0s - loss: 23.0640 - 104ms/epoch - 5ms/step
Epoch 392/500
23

(33.439567933745295,
 51.728773611889906,
             0          1
 0   32.018684  51.017579
 1   33.182445  48.044476
 2   32.082585  51.696062
 3   32.098086  48.209129
 4   26.166071  38.002100
 5   32.642641  50.135069
 6   39.660562  59.372285
 7   38.895034  57.280325
 8   37.504427  54.612408
 9   35.204877  55.828779
 10  23.464857  31.742784
 11  24.114259  33.451257
 12  30.593770  45.176720
 13  30.910087  44.000334
 14  33.081446  50.995177
 15  43.588954  77.001462
 16  35.936694  55.753865
 17  35.484389  54.449751
 18  32.788691  50.194652
 19  33.567462  52.218651
 20  33.339961  51.927814
 21  31.193060  48.558198
 22  38.791248  55.872565
 23  35.817698  57.619072,
 array([[50.00153 , 45.566196, 43.32552 , ..., 52.406166, 53.3579  ,
         50.07719 ],
        [42.867153, 40.088497, 38.112774, ..., 46.359173, 46.48739 ,
         44.38864 ],
        [44.079556, 41.329533, 39.3185  , ..., 48.83313 , 48.98045 ,
         46.18838 ],
        ...,
        [53.702435, 49.7